In [21]:
import pandas as pd
import numpy as np
import scipy as sp
import plotly.plotly as py
import plotly.figure_factory as ff

In [23]:
df = pd.read_csv("school_data.csv")
table = ff.create_table(df)
py.iplot(table, filename='jupyter/table1')

PlotlyRequestError: Hi there! Free users cannot create folders.

To save this file on Plotly, please remove the folder in your filename path.

If you would like to create folders and save files in them, please upgrade your account at: https://goo.gl/NrPpg6 

In [25]:
import plotly.plotly as py
from plotly.graph_objs import *

trace0 = Scatter(
    x=[1, 2, 3, 4],
    y=[10, 15, 13, 17]
)
trace1 = Scatter(
    x=[1, 2, 3, 4],
    y=[16, 5, 11, 9]
)
data = Data([trace0, trace1])

py.iplot(data, filename = 'basic-line')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~estes.zeke/0 or inside your plot.ly account where it is named 'basic-line'


In [1]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

print __version__ # requires version >= 1.9.0

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-1-ae054a2ee653>, line 4)